In [1]:
#import modules
import xarray as xr
import numpy as np
import pandas as pd
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.shapereader as shpreader


import warnings
warnings.filterwarnings('ignore')

/usr/lib/python3.6/site-packages/dask/config.py:129: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
# Import data and merge
datapath='/scratch/local/s1878599/outputs/BB_season2017_16cores_rebus_intel/wrfout_*'
ds = xr.open_mfdataset(datapath)

OSError: no files to open

In [ ]:
#Function to compute organics species for PM2.5.
def pm25_component_of(dataset, species):
    """
    Sum up the species values for bin sizes (from 1 to 3) making up PM25.
    Add the total contribution of the species to PM25 concentration.
    TODO change units from mixing ratio to ug/m3.

    :param species: name of the chemical species
    :type species: string.
    :return:
    :rtype: xarray DataArray.
    """
    return dataset[species + '_a01'] + dataset[species + '_a02'] + dataset[species + '_a03']

In [ ]:
#Calculating all species that contribute to PM2.5 and convert to ug/m3. 
#According to code in module_mosaic_sumpm.F   subroutine sum_pm_mosaic_vbs0 

#Secondary Inorganic Aerosols (SIA)
ds['so4_pm25'] = pm25_component_of(ds,'so4')/ds['ALT']
ds.so4_pm25['units'] = 'ug m-3'
ds['nh4_pm25'] = pm25_component_of(ds,'nh4')/ds['ALT'] 
ds.nh4_pm25['units'] = 'ug m-3'
ds['no3_pm25'] = pm25_component_of(ds,'no3')/ds['ALT'] 
ds.no3_pm25['units'] = 'ug m-3'

#Secondary Organic Aerosols (SOA)
ds['pm25_bio_soa'] =(pm25_component_of(ds,'biog1_o') + pm25_component_of(ds,'biog1_c'))/ds['ALT']   # biogenic from isoprene and pinenes.
ds.pm25_bio_soa['units']= 'ug m-3'
ds['pm25_bb_soa']  =  pm25_component_of(ds,'smpbb')/ds['ALT']  #biomass burning
ds.pm25_bb_soa['units']= 'ug m-3'
ds['pm25_anthro_soa'] = pm25_component_of(ds,'smpa')/ds['ALT']   #anthropogenic
ds.pm25_anthro_soa['units']= 'ug m-3'
ds['pm25_gly_soa'] = pm25_component_of(ds,'glysoa_sfc')/ds['ALT']  #glyoxal
ds.pm25_gly_soa['units']= 'ug m-3'
ds['pm25_soa'] = (ds['pm25_bio_soa'] + ds['pm25_bb_soa'] + ds['pm25_anthro_soa']+ ds['pm25_gly_soa']) #total PM2.5 SOA
ds.pm25_soa['units']= 'ug m-3'

#Black carbon organic carbon BC OC
ds['oc_pm25'] = pm25_component_of(ds,'oc')/ds['ALT'] 
ds.oc_pm25['units']= 'ug m-3'
ds['bc_pm25'] = pm25_component_of(ds,'bc')/ds['ALT'] 
ds.bc_pm25['units']= 'ug m-3'

#Other contributions
ds['pm25_dust'] = pm25_component_of(ds,'oin')/ds['ALT']   #dust
ds.pm25_dust['units']= 'ug m-3'
ds['pm25_seas'] = (pm25_component_of(ds,'na') + pm25_component_of(ds,'cl'))/ds['ALT']   #sea salt
ds.pm25_seas['units']= 'ug m-3'

#Total contribution
ds['pm25_tot_dry'] = ds['pm25_soa'] + ds['oc_pm25'] + ds['bc_pm25'] + ds['so4_pm25'] +ds['nh4_pm25'] + ds['no3_pm25'] +ds['pm25_dust']+ ds['pm25_seas'] 
ds.pm25_tot_dry['units']= 'ug m-3'

In [ ]:
#Defining functions for analysis

def var_space_mean(ds):
 """
  Make the average over 'xlat and xlong' dimension of a datarray.

  :param da:
    datarray to be averaged.
  :type da: xarray DataArray.
  :return:
    Time averaged da.
  :rtype: xarray DataSet.
 """
 return xr.Dataset(dict(ds.mean(dim= ['south_north','west_east'],keep_attrs=True).data_vars), coords=dict(ds.coords))

def timeavg(ds):
    """
      Make the average over 'time' dimension of a datarray.
    """
    return xr.Dataset(dict(ds.mean(dim='Time', keep_attrs=True).data_vars), coords = dict(ds.coords))


def space_subset(dataset, lat_lim, long_lim ):
    """
    Extract spatial subset of data given lat and long limits.
    """
    
    s_subset=ds.where((long_lim[0] < ds.XLONG) & (ds.XLONG < long_lim[1]) & (lat_lim[0] < ds.XLAT) & (ds.XLAT < lat_lim[1]), drop=True)
  
    return s_subset

In [ ]:
#Defining functions for plot

def var_plot_2D(dataset, var_name, level, title):
    
    var = dataset[var_name]
    
    #draw map
    ax = plt.subplot(projection=ccrs.PlateCarree())
    
    # draw coastlines and borders
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, lw=0.5)
    
    
    # Add data from Global Administrative Areas Dataset GDAM https://gadm.org/index.html
    fname = '/exports/csce/datastore/geos/users/s1878599/python_scripts/WRFChemToolkit/geo_data/gadm36_IND_shp/gadm36_IND_1.shp'   

    provinces = list(shpreader.Reader(fname).geometries())
    ax.add_geometries(provinces, ccrs.PlateCarree(),
                      edgecolor='black', facecolor='none', alpha=0.5,
                      lw=0.5)
 
    #draw meridians and parallels
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='k', alpha=0.4, linestyle='-')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 10, 'color': 'gray'}
    gl.ylabel_style = {'size': 10, 'color': 'grey'}
    
    #plot SURFACE data
    long = dataset.XLONG.values[0,:,:]
    lat  = dataset.XLAT.values[0,:,:]
    var_values= var[level,:,:]
    res = 15 #controls the resolution for the map plotting.
    
    cs=plt.contourf( long, lat, var_values,res,
             transform=ccrs.PlateCarree(), cmap=plt.cm.YlGnBu)

    #colorbar
    cbar = plt.colorbar(cs) # pad=0.2 #format='%.2e'
    #cbar.set_label(var.units)
    ax.set_title(title)
    
    #plotting parameters
    plt.rcParams['figure.figsize'] = [10, 5]
    

In [ ]:
#create dates array.
dates = pd.DatetimeIndex(ds.XTIME.values)

In [ ]:
# Space average over the domain
spat_avg = var_space_mean(ds)

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y= spat_avg.pm25_bio_soa[:,0].values,
                    mode='lines',
                    name='SOA biogenic'))

fig.add_trace(go.Scatter(x=dates, y= spat_avg.pm25_bb_soa[:,0].values,
                    mode='lines',
                    name='SOA biomass burning'))

fig.add_trace(go.Scatter(x=dates, y= spat_avg.pm25_anthro_soa[:,0].values,
                    mode='lines',
                    name='SOA anthropogenic'))

fig.add_trace(go.Scatter(x=dates, y= spat_avg.pm25_gly_soa[:,0].values,
                    mode='lines',
                    name='SOA glyoxal'))

fig.add_trace(go.Scatter(x=dates, y= spat_avg.pm25_soa[:,0].values,
                    mode='lines',
                    name='SOA total'))

fig.add_trace(go.Scatter(x=dates, y= spat_avg.so4_pm25[:,0].values,
                    mode='lines',
                    name='SIA sulfate so4'))

fig.add_trace(go.Scatter(x=dates, y= spat_avg.nh4_pm25[:,0].values,
                    mode='lines',
                    name='SIA ammonium nh4'))

fig.add_trace(go.Scatter(x=dates, y= spat_avg.no3_pm25[:,0].values,
                    mode='lines',
                    name='SIA nitrate no3'))


#fig.add_trace(go.Scatter(x=dates, y= spat_avg.bc_pm25[:,0].values,
                    #mode='lines',
                    #name='BC black carbon'))

#fig.add_trace(go.Scatter(x=dates, y= spat_avg.oc_pm25[:,0].values,
                    #mode='lines',
                    #name='POA (OC) primary organic aerosols'))

#fig.add_trace(go.Scatter(x=dates, y= spat_avg.pm25_dust[:,0].values,
                    #mode='lines',
                    #name='Dust'))

#fig.add_trace(go.Scatter(x=dates, y= spat_avg.pm25_seas[:,0].values,
                    #mode='lines',
                    #name='Sea Salt'))

#fig.add_trace(go.Scatter(x=dates, y= spat_avg.pm25_tot_dry[:,0].values,
                    #mode='lines',
                    #name='PM2.5 total (calculated)'))
# Edit the layout
fig.update_layout(title='surface PM2.5 composition contribution from Secondary Aerosols- ALL DOMAIN',
                   xaxis_title='date',
                   yaxis_title='ug/m3')

In [ ]:
# Check that output variable and handcalculated from individual components match.

fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y= spat_avg.PM2_5_DRY[:,0].values,
                    mode='lines',
                    name='PM2_5_DRY'))

fig.add_trace(go.Scatter(x=dates, y= spat_avg.pm25_tot_dry[:,0].values,
                    mode='lines',
                    name='Pm2.5 calculated'))
# Edit the layout
fig.update_layout(title='PM2.5 from model and calcualted from sum of individual contributions ug/m3',
                   xaxis_title='date',
                   yaxis_title='ug/m3')

In [ ]:
#Extrapolating the subset for NCR + BB region.
NCR = space_subset(ds,[28,33], [73.5,78])

In [ ]:
NCR_space_avg = var_space_mean(NCR)

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.pm25_bio_soa[:,0].values,
                    mode='lines',
                    name='SOA biogenic'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.pm25_bb_soa[:,0].values,
                    mode='lines',
                    name='SOA biomass burning'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.pm25_anthro_soa[:,0].values,
                    mode='lines',
                    name='SOA anthropogenic'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.pm25_gly_soa[:,0].values,
                    mode='lines',
                    name='SOA glyoxal'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.pm25_soa[:,0].values,
                    mode='lines',
                    name='SOA total'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.so4_pm25[:,0].values,
                    mode='lines',
                    name='SIA sulfate so4'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.nh4_pm25[:,0].values,
                    mode='lines',
                    name='SIA ammonium nh4'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.no3_pm25[:,0].values,
                    mode='lines',
                    name='SIA nitrate no3'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.bc_pm25[:,0].values,
                    mode='lines',
                    name='BC black carbon'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.oc_pm25[:,0].values,
                    mode='lines',
                    name='POA (OC) primary organic aerosols'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.pm25_dust[:,0].values,
                    mode='lines',
                    name='Dust'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.pm25_seas[:,0].values,
                    mode='lines',
                    name='Sea Salt'))

fig.add_trace(go.Scatter(x=dates, y= NCR_space_avg.pm25_tot_dry[:,0].values,
                    mode='lines',
                    name='PM2.5 total (calculated)'))
# Edit the layout
fig.update_layout(title='surface PM2.5 composition contributions- NCR subset',
                   xaxis_title='date',
                   yaxis_title='ug/m3')

In [ ]:
ds['SIA_pm25'] = ds['no3_pm25'] + ds['so4_pm25'] + ds['nh4_pm25']

In [ ]:
time_avg = timeavg(ds)

In [ ]:
var_plot_2D(time_avg, 'PM2_5_DRY', 0, 'surface PM2.5 average 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'pm25_soa', 0, 'surface SOA (pm2.5) 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'oc_pm25', 0, 'surface POA (pm2.5) 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'bc_pm25', 0, 'surface bc (pm2.5) 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'pm25_bb_soa', 0, 'surface SOA from biomasss burning (pm2.5) 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'pm25_anthro_soa', 0, 'surface SOA from anthro emissions (pm2.5) 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'pm25_bio_soa', 0, 'surface SOA from biog emissions (pm2.5) 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'pm25_gly_soa', 0, 'surface SOA from glyoxal emissions (pm2.5) 11/29 Oct 2019')

In [ ]:
vertical_profile =var_space_mean(time_avg)

In [ ]:
var_plot_2D(time_avg, 'pm25_dust', 0, 'surface dust emissions (pm2.5) 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'pm25_seas', 0, 'surface seas emissions (pm2.5) 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'SIA_pm25', 0, 'surface SIA emissions (pm2.5) 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'co', 0, 'surface CO 11/29 Oct 2019')

In [ ]:
var_plot_2D(time_avg, 'no', 0, 'surface NO 11/29 Oct 2019')

In [ ]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x= vertical_profile.PM2_5_DRY, y=vertical_profile.P_HYD*0.01,
                    mode='lines',
                    ))
# Edit the layout
fig.update_layout(title='Vertical profile',
                   xaxis_title='PM2_5_DRY [ug/m3]',
                   yaxis_title='Pressure hPa')